Q:
 根据症状推理疾病
 根据疾病反推可能的症状

目前手上的模型：
NER： 实体识别，可以分割出语句中的疾病名称和对应的症状，身体部位
RE： 关系抽取，描述实体和对应症状的 关系

给定句子，直接输出Embedding， Gan 根据Cos相似度 # 中文疾病名称归一化

实体对齐
pytorch 计算向量相似度：
https://blog.csdn.net/rocking_struggling/article/details/127855028

# 实体转为Embedding ，然后拉进相似度
Bert 的词向量分布存在问题。
Bert的词向量的分布，会受到词的频率，假定BERT句向量已经包含了足够的语义（比如可以重构出原句子），那么如果它用公式(1)
算余弦值来比较句子相似度时表现不好，那么原因可能就是此时的句向量所属的坐标系并非标准正交基。

https://zhuanlan.zhihu.com/p/374230720
https://spaces.ac.cn/archives/8069

In [22]:
import numpy as np # 基于奇异值分解（SVD） 降维的相似度计算
import scipy
def compute_kernel_bias_svd(vecs, n_components=256):
    """
    参考文献： https://spaces.ac.cn/archives/8069
    SVD 奇异值分解：https://zhuanlan.zhihu.com/p/29846048 有误
    https://www.zhihu.com/tardis/bd/art/26306568?source_id=1001 可学
    计算kernel和bias
    vecs.shape = [num_samples, embedding_size]，
    最后的变换：y = (x + bias).dot(kernel)
    """
    mu = vecs.mean(axis=0, keepdims=True)
    cov = np.cov(vecs.T)
    u, s, vh = np.linalg.svd(cov)
    W = np.dot(u, np.diag(1 / np.sqrt(s)))
    return W[:, :n_components], -mu

def compute_kernel_bias(vecs):
    """计算kernel和bias
    vecs.shape = [num_samples, embedding_size]，
    最后的变换：y = (x + bias).dot(kernel)
    """
    mu = vecs.mean(axis=0, keepdims=True)
    cov = np.cov(vecs.T)
    u, s, vh = np.linalg.svd(cov)
    W = np.dot(u, np.diag(1 / np.sqrt(s)))
    return W, -mu



def compute_svd_torch(vecs, n_components=256):
    """
    参考文献： https://spaces.ac.cn/archives/8069
    SVD 奇异值分解：https://zhuanlan.zhihu.com/p/29846048 有误
    https://www.zhihu.com/tardis/bd/art/26306568?source_id=1001 可学
    计算kernel和bias
    vecs.shape = [num_samples, embedding_size]，
    最后的变换：y = (x + bias).dot(kernel)
    """
    mu = vecs.mean(axis=0, keepdims=True)
    cov = np.cov(vecs.T)
    u, s, vh = np.linalg.svd(cov)
    W = np.dot(u, np.diag(1 / np.sqrt(s)))
    return W[:, :n_components], -mu

In [23]:
import torch
import torch.nn.functional as F

# 定义两个张量
a = torch.tensor([[1,2,3],[2,2,2]],dtype=torch.float32)
b = torch.tensor([[3,3,3],[1,1,1]],dtype=torch.float32)

# 计算cos相似度
cos_sim = F.cosine_similarity(a, b)
print(cos_sim) # tensor([0.0089, 0.4611, 0.1085])


tensor([0.9258, 1.0000])


torch.svd函数返回三个张量：左奇异向量（矩阵U），奇异值（一维张量S）和右奇异向量（矩阵V）的转置。

In [24]:
import torch



In [25]:
# vecs = a
# mu = vecs.mean(dim=0, keepdims=True)
# cov = torch_cov(vecs.T)
# u, s, vh = torch.svd(cov)
# sqrt_inv_s = torch.diag(1 / torch.sqrt(s))
# W = torch.matmul(u, sqrt_inv_s)


基于文本生成的范式
从疾病描述-》疾病实体

训练逻辑思路-》
1. 给定一个句子，句子 -》 embedding 向量 （对均值方差归一化） -》 latent 变换分布映射
2. latent 转为到疾病实体向量空间
3. 标签实体 -》 embedding 向量（均值方差归一化）
4. 计算Cos相似度， 作为监督loss

推理：
句子-》embedding-》白化-》latent变换-》实体向量空间-》vocab》 生成文本
可行性：偏低（无人做过）
预估效果：高
技术难度：高

症状预测疾病：
https://github.com/anujdutt9/Disease-Prediction-from-Symptoms

基于分类范式：
逻辑-》
1. 所有的症状构建为特征， 采用onehot 编码 # 采用bert embedding向量？
2. 所有的疾病构建为类别， 采用onehot 编码
3. 建立随机森林模型，初步判断，逻辑是否可行
4. 后续可能可以转为深度学习方法

推理：
句子 -》提取出疾病症状-》分类
可行性：高
预估效果：可
技术难度：中等偏下
逻辑上存在的问题：
1. 无法包含所有的疾病，只能预测数据集中存在的
2. 可能存在过拟合的现象

In [26]:

import torch
def torch_cov(matrix):
    # 计算矩阵每一列的均值
    mean = torch.mean(matrix, dim=0)
    # 中心化矩阵（减去均值）
    centered_matrix = matrix - mean
    # 计算协方差矩阵
    cov_matrix = torch.matmul(centered_matrix.t(), centered_matrix) / (centered_matrix.size(0) - 1)

    return cov_matrix



def compute_corrcoef(x, y):
    """Spearman相关系数
    """
    return scipy.stats.spearmanr(x, y).correlation

def compute_kernel_bias_svd(vecs, n_components=1):
    """
    参考文献： https://spaces.ac.cn/archives/8069
    SVD 奇异值分解：https://zhuanlan.zhihu.com/p/29846048 有误
    https://www.zhihu.com/tardis/bd/art/26306568?source_id=1001 可学
    计算kernel和bias
    vecs.shape = [num_samples, embedding_size]，
    最后的变换：y = (x + bias).dot(kernel)
    """
    vecs = np.array(vecs,dtype=np.float32)
    mu = vecs.mean(axis=0, keepdims=True)
    print(mu)
    cov = np.cov(vecs.T)
    print(cov)
    u, s, vh = np.linalg.svd(cov)
    W = np.dot(u, np.diag(1 / np.sqrt(s)))
    return W[:, :n_components], -mu

def compute_svd_torch(vecs, n_components=1):
    vecs = torch.tensor(vecs, dtype=torch.float32)
    mu = torch.mean(vecs, dim=0, keepdim=True)
    print(mu)
    # cov = torch.matmul(vecs.t(), vecs) / vecs.size(0)
    cov = torch_cov(vecs.T)
    print(cov)
    u, s, vh = torch.svd(cov)
    sqrt_inv_s = torch.diag(1 / torch.sqrt(s))
    W = torch.matmul(u, sqrt_inv_s)
    W = W[:, :n_components]

    # 返回结果
    return W, -mu

# 示例输入数据
vecs = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]

# 调用compute_svd_torch函数计算结果
W, bias = compute_svd_torch(vecs)
W_np,bias_np = compute_kernel_bias_svd(vecs)
print(W,)
print(W_np)


tensor([[4., 5., 6.]])
tensor([[1., 1., 1.],
        [1., 1., 1.],
        [1., 1., 1.]])
[[4. 5. 6.]]
[[9. 9. 9.]
 [9. 9. 9.]
 [9. 9. 9.]]
tensor([[-0.3333],
        [-0.3333],
        [-0.3333]])
[[-0.11111111]
 [-0.11111111]
 [-0.11111111]]


C:\Users\11958\AppData\Local\Temp\ipykernel_11496\3539555315.py:34: RuntimeWarning: divide by zero encountered in divide
  W = np.dot(u, np.diag(1 / np.sqrt(s)))
